# Erklärung SlidingWindows

Dieser Abschnitt erklärt die meisten Methoden des Objektes SlidingWindows. Weggelassen wurden Methoden, welche nur für die Entwicklung benötigt wurden und im Produktivcode nicht aufgerufen werden (Debugging Methoden).
### Disclaimer
Der Code ist nicht lauffähig, dafür existieren die Python Skripte. Das Notebook stellt lediglich eine Dokumentation dar.

### Verbesserungen, welche an SlidingWindows vorgenommen wurden

- Wenn eine Spur erkannt wurde, wird der Punkt, an welchem sie beginnt gespeichert. Dieser Punkt wird dann im nächsten Frame als Ausgangspunkt für die Spurensuche genommen. Dadurch wird nicht mehr eine ganze Bildhälfte durchsucht, sondern nur das unmittelbare Umfeld des im letzten Frame gefundenen Punktes. Dies spart Rechenkapazität ein und sorgt für genauere und schnellere Ergebnisse. Dies ist genauer beschrieben in set_context.
- Die Windows werden nach oben hin immer breiter. Dadurch kann bei krummen Verlaufen die Spur weiter in der Distanz erkannt werden. Die Windows alle breit zu machen zieht dabei unnötige Rechenkapazität. Beschrieben ist dies genauer in generate_window

<img src="Images/Sliding_windows_bad.png" width="500">

<img src="Images/Sliding_windows_really_good.png" width="500">

### Imports

Dies sind die spezifischen Imports für SlidingWindows. 
Wichtig sind hier die perspective transform als per und preprocess als pre

In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import time
import json
import os

import calib as cal
import perspective_transform as per
import preprocess as pre

### Class SlidingWindow mit init Funktion

Es werden benötigte Objekte (wie Transformation und Preprocess) initialisiert. die last_draw_info und last_frames werden für die unten aufgeführte Spurverfolgung benötigt. Es wird also nicht in jedem Frame die Spur neu gesucht, sondern versucht, die vorher bereits erkannte Spur zu verfolgen.

In [ ]:
class SlidingWindow():
    
    def __init__(self, debug = False, debug_plots = False) -> None:
        """Constructor for the SlidingWindow class

        Args:
            debug (bool, optional): Debug mode. Defaults to False.
            debug_plots (bool, optional): Debug mode with plot of histogram. Defaults to False.
        """
        
        self.transformation = per.Transformation(debug)
        self.pre = pre.Preprocess()
        self.debug = debug
        self.debug_plots = debug_plots
        self.loaded = False   
        
        # Remember last frames and draw information
        self.last_draw_info = None
        self.last_frame_right_x = None
        self.last_frame_left_x = None


### load config
Hier werden spezifische Konfigurationen für Sliding Window aus der Konfigurationsdatei geladen und auf Vollständigkeit überprüft. Es gibt zwei verschiedene Konfigurationen, eine für das default Video, eine für das Fortgeschrittene.
Wenn alle Felder gefunden wurden, werden sie als Attribute dem Objekt zugewiesen.

In [ ]:
def load_config(self, path):
    if not os.path.exists(path):
        return print('File '+ path +' not found')
    
    with open(path, 'r') as f:
        config = json.load(f)
    
    if not config:
        return 'Error: Config not found'
    if not 'SLIDING_WINDOWS' in config.keys():
        return 'Error: SLIDING_WINDOWS is missing'
    if not 'N_WINDOWS' in config['SLIDING_WINDOWS'].keys():
        return 'Error: N_WINDOWS is missing'
    if not 'MARGIN' in config['SLIDING_WINDOWS'].keys():
        return 'Error: MARGIN is missing'
    if not 'MIN_PIX' in config['SLIDING_WINDOWS'].keys():
        return 'Error: MIN_PIX is missing'
    if not 'THRESH' in config['SLIDING_WINDOWS'].keys():
        return 'Error: THRESH is missing'
    if not 'LANE_WIDTH_FOR_SEARCH' in config['SLIDING_WINDOWS'].keys():
        return 'Error: LANE_WIDTH_FOR_SEARCH is missing'
    if not 'SCALING_OF_BOX_WIDTH' in config['SLIDING_WINDOWS'].keys():
        return 'Error: SCALING_OF_BOX_WIDTH is missing'
    if not 'MIN_COLOR' in config['SLIDING_WINDOWS'].keys():
        return 'Error: MIN_COLOR is missing'
    if not 'MAX_COLOR' in config['SLIDING_WINDOWS'].keys():
        return 'Error: MAX_COLOR is missing'
    if not 'TRANS_MATRIX' in config['SLIDING_WINDOWS'].keys():
        return 'Error: TRANS_MATRIX is missing'
    if not 'HIT_X_LEFT' in config['SLIDING_WINDOWS'].keys():
        return 'Error: HIT_X_LEFT is missing'
    if not 'HIT_Y_LEFT' in config['SLIDING_WINDOWS'].keys():
        return 'Error: HIT_Y_LEFT is missing'
    if not 'HIT_X_RIGHT' in config['SLIDING_WINDOWS'].keys():
        return 'Error: HIT_X_RIGHT is missing'
    if not 'HIT_Y_RIGHT' in config['SLIDING_WINDOWS'].keys():
        return 'Error: HIT_Y_RIGHT is missing'
    
    self.n_windows = config['SLIDING_WINDOWS']['N_WINDOWS']
    self.margin = config['SLIDING_WINDOWS']['MARGIN']
    self.min_pix = config['SLIDING_WINDOWS']['MIN_PIX']
    self.thresh = config['SLIDING_WINDOWS']['THRESH']
    self.lane_width_for_search = config['SLIDING_WINDOWS']['LANE_WIDTH_FOR_SEARCH']
    self.scaling_of_box_width = config['SLIDING_WINDOWS']['SCALING_OF_BOX_WIDTH']
    self._min_color = config['SLIDING_WINDOWS']['MIN_COLOR']
    self._max_color = config['SLIDING_WINDOWS']['MAX_COLOR']
    self.trans_matrix = config['SLIDING_WINDOWS']['TRANS_MATRIX']
    self._hit_x_left = config['SLIDING_WINDOWS']['HIT_X_LEFT']
    self._hit_y_left = config['SLIDING_WINDOWS']['HIT_Y_LEFT']
    self._hit_x_right = config['SLIDING_WINDOWS']['HIT_X_RIGHT']
    self._hit_y_right = config['SLIDING_WINDOWS']['HIT_Y_RIGHT']
    
    self.loaded = True
    
    return None

### execute
Diese Methode stellt die 'Main' der Klasse dar. Um SlidingWindows von außerhalb zu starten, muss diese Methode ausgeführt werden. Sie ruft dann die jeweiligen Hilfsfunktionen auf. 
Der grobe Ablauf ist:
- Das Bild vorverarbeiten (Mit versch. Filtern und Transformation in Vogelperspektive)
- Es werden Suchbereiche für die Fahrspuren mit dem Histogram festgelegt
- Die einzelnen Windows werden erstellt und angewendet
- Die Koordinaten für die Fahrspur werden berechnet
- Wenn das Ergebnis plausibel ist, dann einzeichnen der Fahrspur in das ursprüngliche Bild

Anschließend wird das Bild zurückgegeben. Falls das Ergebnis nicht plausibel ist, wird das ursprüngliche Bild zurückgegeben.

In [ ]:
def execute(self, img):
    """Execute the sliding window algorithm

    Args:
        img (Image): Current frame

    Returns:
        Image: Processed frame
    """
    if not self.loaded:
        return False
    
    # Preprocess the image
    img_transformed, M_reverse = self._preprocess(img)

    if self.debug:
        cv.imshow('transformed', img_transformed)
        
            
    # Set local vars
    hist = self.get_histogram(img_transformed)
    img_y_shape = img_transformed.shape[0]
    img_x_shape = img_transformed.shape[1]
    self.set_context(img_transformed, hist)

    # Generate the windows
    for i_window in range(self.n_windows):
        self._generate_window(img_y_shape, i_window)

    # Get the drawing information
    draw_info = self._generate_line_coordinates(img_y_shape, img_x_shape)

    if not draw_info:

        if not self.last_draw_info:
            return img
        else:
            draw_info = self.last_draw_info
    
    else:
        self.last_draw_info = draw_info

    # Draw the line
    if self.check_plausibility(draw_info, img.shape): img = self._draw_lane_area(img, img_transformed, M_reverse, draw_info)

    # Return finished frame
    return img     

### check plausibility

Überprüft die erhaltenen Spuren. Wenn sich eine Koordinate der Spur im einer der beiden unteren Ecken befindet oder sich die Polynome (im Sichtbereich) schneiden, wird das Ergebnis als nicht gültig angesehen. Die Größe der Boxen kommt auf das Video an, also wie nahe das Fahrzeug an welcher Spur fährt. 
Wenn sich die Spuren kreuzen weicht mindestens eine von der reellen Spur ab (Fahrspuren kreuzen sich in den allermeisten Fällen nicht). Außerdem ist beim geradeaus fahren innerhalb einer Spur auf einem Highway keine Spur in den Ecken zu erwarten, was das Ergebnis also auch unplausibel macht.

In [ ]:
def check_plausibility(self, draw_info, img_shape) -> bool:
    y_values = draw_info['PLOT_Y']
    left_fit_x = draw_info['LEFT_FIT_X']
    right_fit_x = draw_info['RIGHT_FIT_X']
    
    # Check for the hit boxes
    if any(left_fit_x[self._hit_y_left:] <= self._hit_x_left):
        return False
    
    if any(right_fit_x[self._hit_y_right:] >= img_shape[1] + self._hit_x_right):
        return False
    
    # Check for crossing lines
    if any(left_fit_x >= right_fit_x):
        return False
    
    return True

### preprocess
Zuerst wird jeder Gelb-Wert auf weiß gemappt. Dies wird gemacht, um nachher den Threshold besser einstellen zu können. Gelb ist in der Grau-Darstellung dunkler als weiß, dadurch konnte der Threshold nicht präzise genug eingestellt werden und es wurden viele Störsignale erkannt. Mithilfe dieser Transformation der gelben Farbwerte konnte die Threshold Vorverarbeitung deutlich verbessert werden, was zu einer deutlichen Präzisionssteigerung der Erkennung geführt hat.

Anschließend wird das Bild in Graustufe konvertiert und ein Gauss Filter angewandt. Das Bild wird dann in Vogelperspektive transformiert und anschließend wird noch der Threshold darauf angewandt. Es werden das vorverarbeitete Bild und die Matrix zur Rückumwandlung zurückgegeben. Letztere wird später zur Darstellung der erkannten Spuren verwendet.

In [ ]:
def _preprocess(self, img):
    """Preprocess the image, apply filters and transformations

    Args:
        img (Image): Current Frame

    Returns:
        Tuple: Transformed image and the reversed transformation matrix
    """
    # Find the yellow line
    if self._min_color and self._max_color:
        img = self.pre.map_color(img, self._min_color, self._max_color)
        if self.debug:
            cv.imshow('yellow', img)
    
    # Convert to grayscale
    img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    
    # Apply gaussian blur
    img = self.pre.gauss(img)
    
    # Threshold the image to areal view
    img_transformed, M_reversed = self.transformation.transform_image_perspective(img, self.trans_matrix)
    
    # Apply threshold
    img_transformed = self.pre.threshold(img_transformed, self.thresh)

    return img_transformed, M_reversed

### get histogram
Generiert das Histogram der unteren Hälfte eines Bildes. Dies hat den Zweck, Störungen im oberen Bereich nicht zu beachten. Enthält im Produktivcode noch Debugging Möglichkeiten.

In [ ]:
def get_histogram(self, img):
    """Generate the histogram of the transformed image

    Args:
        img (Image): Current Frame

    Returns:
        List: Histogram of the image
    """
    # Get the histogram of the image
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)

    return histogram

### set context
In dieser Methode wird der Bereich des Bildes für die eigentliche Erkennung der Spur definiert. Dies ist vor allem davon abhängig, ob im letzten Frame eine Spur erkannt wurde, welche nun verfolgt werden kann, oder ob die Spur neu gesucht werden muss. Es wird dabei zwischen linker und rechter Spur unterschieden, da es möglich ist, nur eine der beiden Spuren zu erkennen / zu speichern. Wenn keine Spur aus dem letzten Frame gespeichert wurde, wird das Bild mittig geteilt und der Suchbereich ist die komplette Hälfte. Wenn die Position der Spur aus dem letzten Frame gespeichert wurde, wird der Bereich um diesen X Wert ausgewählt. Das Offset für diese Grenzen um den X Wert wird in der Konfiguration gespeichert und in lane_width_for_search gespeichert. Default-mäßig liegt dieser Wert bei 10 Pixeln. Damit wird der Suchbereich der Spur von Halbes Bild (ca 700 Pixel) auf das 10 Pixel Umfeld der zuletzt erkannten Spur eingeschränkt (also 20 Pixel). Mit dieser Methode wird Rechenkapazität eingespart.

In [ ]:
def set_context(self, img, hist):
    """Generate the search context for the windows (based on the previous frame)

    Args:
        img (Image): Current Frame
        hist (List): Histogram of the current frame
    """
    # Generate black layered image
    mid = img.shape[1]//2

    if not self.last_frame_left_x:
        # Divide the histogram into two parts
        leftx_base = np.argmax(hist[:mid])
    else:
        left_negative = self.last_frame_left_x - self.lane_width_for_search
        left_positive = self.last_frame_left_x + self.lane_width_for_search
        if left_negative < 1:
            left_negative = 1
            left_positive = mid
        leftx_base = np.argmax(hist[left_negative : left_positive]) + left_negative
        self.last_frame_left_x = None
        
    if not self.last_frame_right_x:
        # Divide the histogram into to parts
        rightx_base = np.argmax(hist[mid:]) + mid
    else:
        right_negative = self.last_frame_right_x - self.lane_width_for_search
        right_positive = self.last_frame_right_x + self.lane_width_for_search
        if right_positive > img.shape[1] - 1:
            right_negative = mid
            right_positive = img.shape[1] - 1 
        rightx_base = np.argmax(hist[right_negative : right_positive]) + right_negative
        self.last_frame_right_x = None


    # Number of sliding windows in the frame
    # self.n_windows = 10
    self.window_height = img.shape[0]//self.n_windows

    # Find coordinates which are not zero
    nonzero = img.nonzero()
    self.nonzeroy = np.array(nonzero[0])
    self.nonzerox = np.array(nonzero[1])

    self.current_leftx = leftx_base
    self.current_rightx = rightx_base

    self.left_lane_inds = []
    self.right_lane_inds = []

### generate window
Diese Funktion wird in execute in einer Schleife aufgerufen, für jede window Position einzeln. Die Methode generiert also für den angegebenen Suchbereich ein Window und positioniert es an der Stelle, wo sie etwas erkannt hat. Sie speichert die Position des untersten Frames (also die Position der Spurmarkierung am nächsten beim Auto). Die Indices werden in den lane_inds Attributen des Objektes gespeichert. Wichtig zu beachten ist hier die Vergrößerung der Windows. Diese werden nach obenhin (also in die Ferne) immer breiter, da der in Vogelperspektive transformierte Kurvenverlauf sich oben deutlich stärker krümmen kann als unten. Ohne diese Vergrößerung konnten Linien nicht so sehr in die Weite erkannt werden, da sie sich oft zu stark krümmten. Die Fenster alle so groß zu machen wie benötigt ist allerdings rechenintensiver. Also wurde diese Fensterbreite linear zunehmend designed.

In [ ]:
def _generate_window(self, img_y_shape, index):
    """Generate the window for the current index

    Args:
        img_y_shape (int): y shape of the image
        index (int): index of the current window
    """
    # Set current window coordinates
    win_y_low = img_y_shape - (index + 1) * self.window_height
    win_y_high = img_y_shape - index * self.window_height
    
    # Define box-window coordinates
    win_xleft_low = self.current_leftx - (self.margin + index * self.scaling_of_box_width)
    win_xleft_high = self.current_leftx + (self.margin + index * self.scaling_of_box_width)
    win_xright_low = self.current_rightx - (self.margin + index * self.scaling_of_box_width)
    win_xright_high = self.current_rightx + (self.margin + index * self.scaling_of_box_width)

    # Get the indices where the coordinates of the image are not
    # zero but in the window (defined by the win_y_low, ...)
    left_inds = ((self.nonzeroy >= win_y_low) & (self.nonzeroy < win_y_high) & (self.nonzerox >= win_xleft_low) & (self.nonzerox < win_xleft_high)).nonzero()[0]
    right_inds = ((self.nonzeroy >= win_y_low) & (self.nonzeroy < win_y_high) & (self.nonzerox >= win_xright_low) & (self.nonzerox < win_xright_high)).nonzero()[0]
    self.left_lane_inds.append(left_inds)
    self.right_lane_inds.append(right_inds)

    # Change the current indices
    if len(left_inds) > self.min_pix:
        self.current_leftx = int(np.mean(self.nonzerox[left_inds]))
        if index == 0:
            self.last_frame_left_x = self.current_leftx
    if len(right_inds) > self.min_pix:
        self.current_rightx = int(np.mean(self.nonzerox[right_inds]))
        if index == 0:
            self.last_frame_right_x = self.current_rightx

### generate line coordinates
Diese Methode generiert aus den erkannten Punkten der generate_windows Methode vollständige Polynome für die Spuren. Wenn Punkte gefunden wurden, werden diese per polyfit() Funktion umgewandelt. Es wird ein dictionary mit allen wichtigen Werten und Funktionsparametern zurückgegeben.

In [ ]:
def _generate_line_coordinates(self, img_y_shape, img_x_shape):
    """Generate the line coordinates for the left and right lane

    Args:
        img_y_shape (int): shape of the image in y direction
        img_x_shape (int): shape of the image in x direction

    Returns:
        Dict: Dictionary with the left and right lane coordinates
    """
    # Flatten array
    self.left_lane_inds = np.concatenate(self.left_lane_inds)
    self.right_lane_inds = np.concatenate(self.right_lane_inds)

    leftx = self.nonzerox[self.left_lane_inds]
    lefty = self.nonzeroy[self.left_lane_inds]
    rightx = self.nonzerox[self.right_lane_inds]
    righty = self.nonzeroy[self.right_lane_inds]

    # Check wether lines are detected:
    if len(leftx) <= 0 or len(lefty) <= 0 or len(rightx) <= 0 or len(righty) <= 0:
        # Prepare return values
        ret = {}
        return ret

    # Create line
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Show plot of the created lines
    plot_y = np.linspace(0, img_y_shape - 1, img_y_shape)
    left_fit_x = left_fit[0] * plot_y**2 + left_fit[1] * plot_y + left_fit[2]
    right_fit_x = right_fit[0] * plot_y**2 + right_fit[1] * plot_y + right_fit[2]

    # Prepare return values
    return {
        'LEFT_X': leftx,
        'RIGHT_X': rightx,
        'LEFT_FIT_X': left_fit_x,
        'RIGHT_FIT_X': right_fit_x,
        'PLOT_Y': plot_y
    }

### draw lane area
Zeichnet die gefundene Fahrspur in das ursprüngliche Video ein. 
Zeichnet hierfür in neues Null-Array (Form von Vogelperspektive-Bild) die Fläche ein und transformiert dann dieses neue Bild zurück in die normale Perspektive. Anschließend wird das neue Bild mit den Markierungen über das eigentliche Image gelegt.

In [ ]:
def _draw_lane_area(self, original_img, transformed_img, M_reversed, draw_info):
    """Draw the lane area on the original image

    Args:
        original_img (Image): original image
        transformed_img (Image): transformed image
        M_reversed (List): reversed transformation matrix
        draw_info (Dict): dictionary with the coordinates of the lane

    Returns:
        Image: image with the lane area
    """
    # Unpack draw Info
    left_fit_x = draw_info['LEFT_FIT_X']
    right_fit_x = draw_info['RIGHT_FIT_X']
    plot_y = draw_info['PLOT_Y']

    transformed_zero = np.zeros_like(transformed_img).astype(np.uint8)
    color_transformed = np.dstack((transformed_zero, transformed_zero, transformed_zero))

    # Generate the points of the lane
    pts_left = np.array([np.transpose(np.vstack([left_fit_x, plot_y]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fit_x, plot_y])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the driving lane in the transformed image
    cv.fillPoly(color_transformed, np.int_([pts]), (0,255,0))

    # Untransform the image back 
    new_transformed = cv.warpPerspective(color_transformed, M_reversed, (original_img.shape[1], original_img.shape[0]))
    result = cv.addWeighted(original_img, 1, new_transformed, 0.3, 0)

    # Show the transformed aria
    if self.debug: cv.imshow("Transformed Aria", new_transformed)

    return result   